In [ ]:
import syft as sy
import numpy as np
from syft import phi_tensor

from syft.core.adp.data_subject_ledger import DataSubjectLedger
from syft.core.adp.ledger_store import DictLedgerStore
from typing import Any
from syft.core.adp.data_subject import DataSubject
import numpy as np

data1 = np.random.randint(0, 100000, size=(5,5))
data2 = np.random.randint(0, 100000, size=(5,5))

pt = phi_tensor.PhiTensor(child=data1, data_subjects=1, min_vals=0, max_vals=1)
pt2 = phi_tensor.PhiTensor(child=data2, data_subjects=2, min_vals=0, max_vals=1)
gt = pt + pt2
gt.state.keys(), gt.data_subjects.data_subjects_indexed

/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "
/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(dict_keys(['1257419458', '819217519']), array(0))

In [ ]:
ledger_store = DictLedgerStore()
# print(ledger_store.kv_store)
user_key = b"1231"
ledger = DataSubjectLedger.get_or_create(store=ledger_store, user_key=user_key)
max_budget = 100

def get_budget_for_user(*args: Any, **kwargs: Any) -> float:
    return 9999999

def deduct_epsilon_for_user(*args: Any, **kwargs: Any) -> bool:
    return True


Creating new Ledger


Phi Tensor

In [ ]:
# Phi Tensor
# with calibration: query_limit = 5
results = pt.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  5
Sigma before Calibration:
  0.1
****** Budget spent EXCEEDS limit set by data owner
Sigma after Calibration: 
 48156.49253216019


  0%|          | 0/1 [00:00<?, ?it/s]/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/jax/interpreters/mlir.py:512: UserWarning: Some donated buffers were not usable: ShapedArray(float64[25]), ShapedArray(int64[]).
Donation is not implemented for cpu.
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn(f"Some donated buffers were not usable: {', '.join(unused_donations)}.\n{msg}")
100%|██████████| 1/1 [00:00<00:00,  9.10it/s]


 Got user budget 9999999.0 epsilon_spent 45.344674707311974


In [4]:
# Phi Tensor
# without calibration: query_limit = 99999
results = pt.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  999999
Sigma before Calibration:
  0.1
****** Budget spent BELOW limit set by data owner
Sigma after Calibration: 
 0.1


100%|██████████| 1/1 [00:00<00:00, 619.91it/s]


 Got user budget 9999999.0 epsilon_spent 706263.403703397


Gamma Tensor

In [4]:
# Gamma Tensor
# with calibration: query_limit = 5

results = gt.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  5
Sigma before Calibration:
  0.1
****** Budget spent EXCEEDS limit set by data owner
Sigma after Calibration: 
 48336.31164042205


100%|██████████| 2/2 [00:00<00:00, 559.46it/s]


 Got user budget 9999999.0 epsilon_spent 45.09789905820535

 Got user budget 9999999.0 epsilon_spent 42.67472557475451


In [6]:
# Gamma Tensor
# without calibration:  query_limit = 999999
results = gt.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  999999
Sigma before Calibration:
  0.1
****** Budget spent BELOW limit set by data owner
Sigma after Calibration: 
 0.1


100%|██████████| 2/2 [00:00<00:00, 780.12it/s]


 Got user budget 9999999.0 epsilon_spent 706263.403703397

 Got user budget 9999999.0 epsilon_spent 706263.403703397


Syft Tensors

In [5]:

entity1 = np.random.choice([DataSubject(name="1")], size=5)
entity2 = np.random.choice( [DataSubject(name="2")], size=5)

pt_tensor = sy.Tensor(np.random.randint(0, 100000, size=(5,5))).private(min_val=0, max_val=2, data_subjects=entity1)
pt_tensor2 = sy.Tensor(np.random.randint(0, 100000, size=(5,5))).private(min_val=0, max_val=2, data_subjects=entity2)
pt_tensor.shape, pt.shape

((5, 5), (5, 5))

In [7]:
# Syft Phi Tensor
# with calibration: query_limit = 5
results = pt_tensor.child.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  5
Sigma before Calibration:
  0.1
****** Budget spent EXCEEDS limit set by data owner
Sigma after Calibration: 
 48341.53464671969


100%|██████████| 1/1 [00:00<00:00, 115.59it/s]


 Got user budget 9999999.0 epsilon_spent 34.318470727337356


In [9]:
# Syft Phi Tensor
# without calibration: query_limit = 999999
results = pt_tensor.child.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  999999
Sigma before Calibration:
  0.1
****** Budget spent BELOW limit set by data owner
Sigma after Calibration: 
 0.1


100%|██████████| 1/1 [00:00<00:00, 266.95it/s]


 Got user budget 9999999.0 epsilon_spent 706263.403703397


Gamma Tensor

In [6]:
# Syft Gamma Tensor
# query_limit = 5
gt_tensor = pt_tensor + pt_tensor2
results = gt_tensor.child.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  5
Sigma before Calibration:
  0.1
****** Budget spent EXCEEDS limit set by data owner
Sigma after Calibration: 
 49038.394396635784


  0%|          | 0/2 [00:00<?, ?it/s]/Users/ivolinengong/miniforge3/envs/syft_env/lib/python3.10/site-packages/jax/interpreters/mlir.py:512: UserWarning: Some donated buffers were not usable: ShapedArray(float64[25]), ShapedArray(int64[5]).
Donation is not implemented for cpu.
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn(f"Some donated buffers were not usable: {', '.join(unused_donations)}.\n{msg}")
100%|██████████| 2/2 [00:00<00:00, 43.45it/s]


 Got user budget 9999999.0 epsilon_spent 33.608318799859106

 Got user budget 9999999.0 epsilon_spent 37.87480848777142


In [11]:
# Syft Gamma Tensor
# query_limit = 99999
gt_tensor = pt_tensor + pt_tensor2
results = gt_tensor.child.publish(
    get_budget_for_user=get_budget_for_user,
    deduct_epsilon_for_user=deduct_epsilon_for_user,
    ledger=ledger,
    sigma=0.1,
)

Query Limit:  999999
Sigma before Calibration:
  0.1
****** Budget spent BELOW limit set by data owner
Sigma after Calibration: 
 0.1


100%|██████████| 2/2 [00:00<00:00, 568.03it/s]


 Got user budget 9999999.0 epsilon_spent 706263.403703397

 Got user budget 9999999.0 epsilon_spent 706263.403703397
